# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 70 new papers today
          32 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/31 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.00029


extracting tarball to tmp_2212.00029...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.00032


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.00029/mnras_guide.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.00775


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.00766/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2212.00775...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.00777


extracting tarball to tmp_2212.00777... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00032-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00032) | **PHANGS-JWST First Results: Spurring on Star Formation: JWST Reveals  Localised Star Formation in a Spiral Arm Spur of NGC 628**  |
|| Thomas G. Williams, et al. -- incl., <mark>Eva Schinnerer</mark>, <mark>Jonathan D. Henshaw</mark>, <mark>Frank Bigiel</mark>, <mark>Kathryn Kreckel</mark>, <mark>Daizhong Liu</mark>, <mark>Francesca Pinna</mark>, <mark>Rowan Smith</mark>, <mark>Sophia Stuber</mark> |
|*Appeared on*| *2022-12-02*|
|*Comments*| *11 pages, 5 Figures, accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ*|
|**Abstract**| We combine JWST observations with ALMA CO and VLT-MUSE H$\alpha$ data to examine off-spiral arm star formation in the face-on, grand-design spiral galaxy NGC 628. We focus on the northern spiral arm, around a galactocentric radius of 3-4 kpc, and study two spurs. These form an interesting contrast, as one is CO-rich and one CO-poor, and they have a maximum azimuthal offset in MIRI 21$\mu$m and MUSE H$\alpha$ of around 40$^\circ$ (CO-rich) and 55$^\circ$ (CO-poor) from the spiral arm. The star formation rate is higher in the regions of the spurs near to spiral arms, but the star formation efficiency appears relatively constant. Given the spiral pattern speed and rotation curve of this galaxy and assuming material exiting the arms undergoes purely circular motion, these offsets would be reached in 100-150 Myr, significantly longer than the 21$\mu$m and H$\alpha$ star formation timescales (both <10 Myr). The invariance of the star formation efficiency in the spurs versus the spiral arms indicates massive star formation is not only triggered in spiral arms, and cannot simply occur in the arms and then drift away from the wave pattern. These early JWST results show that in-situ star formation likely occurs in the spurs, and that the observed young stars are not simply the `leftovers' of stellar birth in the spiral arms. The excellent physical resolution and sensitivity that JWST can attain in nearby galaxies will well resolve individual star-forming regions and help us to better understand the earliest phases of star formation. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00047-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00047) | **A first glimpse at the nuclear stellar disc line-of-sight structure**  |
|| <mark>Francisco Nogueras-Lara</mark> |
|*Appeared on*| *2022-12-02*|
|*Comments*| *Accepted for publication in A&A. 7 pages, 6 figures*|
|**Abstract**| The nuclear stellar disc (NSD) is a dense stellar structure at the centre of our Galaxy. Given its proximity, it constitutes a unique laboratory to understand other galactic nuclei. Nevertheless, the high crowding and extinction hamper its study, and even its morphology and kinematics are not yet totally clear. In this work we use NSD red clump stars, whose intrinsic properties are well known, to trace the kinematics of the NSD and to compute the distance and extinction towards the edges of the NSD. We used publicly available proper motion and photometric catalogues of the NSD to distinguish red clump stars by using a colour-magnitude diagram. We then applied a Gaussian mixture model to obtain the proper motion distribution, and computed the extinction and distance towards stars with different kinematics. We obtained that the proper motion distributions contain NSD stars rotating eastwards and westwards, plus some contamination from Galactic bulge/bar stars, in agreement with previous work. We computed the distance and extinction towards the eastward- and westward-moving stars and concluded that the latter are $\sim300$ pc beyond, indicating a similar structure along and across the line of sight, and consistent with an axisymmetric structure of the NSD. Moreover, we found that the extinction within the NSD is relatively low and accounts for less than 10 % of the total extinction of the stars belonging to the farthest edge of the NSD. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00051-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00051) | **Morphological Parameters and Associated Uncertainties for 8 Million  Galaxies in the Hyper Suprime-Cam Wide Survey**  |
|| <mark>Aritra Ghosh</mark>, et al. |
|*Appeared on*| *2022-12-02*|
|*Comments*| *Submitted to ApJ. Comments welcome. arXiv admin note: text overlap with arXiv:2207.05107*|
|**Abstract**| We use the Galaxy Morphology Posterior Estimation Network (GaMPEN) to estimate morphological parameters and associated uncertainties for $\sim 8$ million galaxies in the Hyper Suprime-Cam (HSC) Wide survey with $z \leq 0.75$ and $m \leq 23$. GaMPEN is a machine learning framework that estimates Bayesian posteriors for a galaxy's bulge-to-total light ratio ($L_B/L_T$), effective radius ($R_e$), and flux ($F$). By first training on simulations of galaxies and then applying transfer learning using real data, we trained GaMPEN with $<1\%$ of our dataset. This two-step process will be critical for applying machine learning algorithms to future large imaging surveys, such as the Rubin-Legacy Survey of Space and Time (LSST), the Nancy Grace Roman Space Telescope (NGRST), and Euclid. By comparing our results to those obtained using light-profile fitting, we demonstrate that GaMPEN's predicted posterior distributions are well-calibrated ($\lesssim 5\%$ deviation) and accurate. This represents a significant improvement over light profile fitting algorithms which underestimate uncertainties by as much as $\sim60\%$. For an overlapping sub-sample, we also compare the derived morphological parameters with values in two external catalogs and find that the results agree within the limits of uncertainties predicted by GaMPEN. This step also permits us to define an empirical relationship between the S\'ersic index and $L_B/L_T$ that can be used to convert between these two parameters. The catalog presented here represents a significant improvement in size ($\sim10 \times $), depth ($\sim4$ magnitudes), and uncertainty quantification over previous state-of-the-art bulge+disk decomposition catalogs. With this work, we also release GaMPEN's source code and trained models, which can be adapted to other datasets. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00765-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00765) | **Multi-collision internal shock lepto-hadronic models for energetic GRBs**  |
|| Annika Rudolph, Maria Petropoulou, Željka Bošnjak, <mark>Walter Winter</mark> |
|*Appeared on*| *2022-12-02*|
|*Comments*| *For application to GRB 221009A see companion paper, comments welcome*|
|**Abstract**| For a sub-population of energetic Gamma-Ray Bursts (GRBs), a moderate baryonic loading may suffice to power Ultra-High-Energy Cosmic Rays (UHECRs). Motivated by this, we study the radiative signatures of cosmic-ray protons in the prompt phase of energetic GRBs. Our framework is the internal shock model with multi-collision descriptions of the relativistic ejecta (with different emission regions along the jet), plus time-dependent calculations of photon and neutrino spectra. Our GRB prototypes are motivated by {\em Fermi}-LAT detected GRBs (including GRB~221009A) for which further, owing to the large energy flux, neutrino non-observation of single events may pose a strong limit on the baryonic loading. We study the feedback of protons on electromagnetic spectra in synchrotron- and inverse Compton-dominated scenarios to identify the multi-wavelength signatures, to constrain the maximally allowed baryonic loading, and to point out the differences between hadronic and inverse Compton signatures. We find that hadronic signatures appear as correlated flux increases in the optical-UV to soft X-ray and GeV to TeV gamma-ray ranges in the synchrotron scenarios, whereas they are difficult to identify in inverse Compton-dominated scenarios. We demonstrate that baryonic loadings around 10, which satisfy the UHECR energetic requirements, do not distort the predicted photon spectra in the {\em Fermi}-GBM range and are consistent with constraints from neutrino data if the collision radii are large enough (i.e., the time variability is not too short). It therefore seems plausible that under the condition of large dissipation radii a population of energetic GRBs can be the origin of the UHECRs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00766-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00766) | **Multi-messenger model for the prompt emission from GRB 221009A**  |
|| Annika Rudolph, Maria Petropoulou, <mark>Walter Winter</mark>, Željka Bošnjak |
|*Appeared on*| *2022-12-02*|
|*Comments*| *9 pages, 3 figures, 1 table. For model details see companion paper, comments welcome*|
|**Abstract**| We present a multi-messenger model for the prompt emission from GRB 221009A within the internal shock scenario. We consider the time-dependent evolution of the outflow with its impact on the observed light curve from multiple collisions, and the self-consistent generation of the electromagnetic spectrum in synchrotron and inverse Compton-dominated scenarios. Our leptohadronic model includes UHE protons potentially accelerated in the outflow, and their feedback on spectral energy distribution and on the neutrino emission. We find that we can roughly reproduce the observed light curves with an engine with varying ejection velocity of ultra-relativistic material, which has an intermediate quiescent period of about 200 seconds and a variability timescale of $\sim1$~s. We consider baryonic loadings of 3 and 30 that are compatible with the hypothesis that the highest-energetic LHAASO photons might come from UHECR interactions with the extragalactic background light, and the paradigm that energetic GRBs may power the UHECR flux. For these values and the high dissipation radii considered we find consistency with the non-observation of neutrinos and no significant signatures on the electromagnetic spectrum. Inverse Compton-dominated scenarios from the prompt emission are demonstrated to lead to about an order of magnitude higher fluxes in the HE-range; this enhancement is testable by its spectral impact in the Fermi-GBM and LAT ranges. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00775-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00775) | **Distinct signatures of spinning PBH domination and evaporation: doubly  peaked gravitational waves, dark relics and CMB complementarity**  |
|| Nilanjandev Bhaumik, <mark>Anish Ghoshal</mark>, Rajeev Kumar Jain, Marek Lewicki |
|*Appeared on*| *2022-12-02*|
|*Comments*| *32 pages, 10 Figures*|
|**Abstract**| Ultra-low mass primordial black holes (PBH) which may briefly dominate the enegy density of the universe but completely evaporate before the big bang nucleosynthesis (BBN), can lead to interesting observable signatures. In our previous work, we studied the generation of a doubly peaked spectrum of induced stochastic gravitational wave background (ISGWB) for such a scenario and explored the possibility of probing a class of baryogenesis models wherein the emission of massive unstable particles from the PBH evaporation and their subsequent decay contributes to the matter-antimatter asymmetry. In this work, we extend the scope of our earlier work by including spinning PBHs and consider the emission of light relativistic dark sector particles, which contribute to the dark radiation (DR) and massive stable dark sector particles, thereby accounting for the dark matter (DM) component of the universe. The ISGWB can be used to probe the non-thermal production of these heavy DM particles, which cannot be accessible in any laboratory searches. For the case of DR, we find a novel complementarity between the measurements of $\Delta N_{\rm eff}$ from these emitted particles and the ISGWB from PBH domination. Our results indicate that the ISGWB has a weak dependence on the initial PBH spin. However, for gravitons as the DR particles, the initial PBH spin plays a significant role, and only above a critical value of the initial spin parameter $a_*$, which depends only on initial PBH mass, the graviton emission can be probed in the CMB-HD experiment. Upcoming CMB experiments such as CMB-HD and CMB-Bharat, together with future GW detectors like LISA and ET, therefore, open up an exciting possibility of constraining the PBHs parameter space and providing deeper insights into the expansion history of the universe between the end of inflation and BBN. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00777-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00777) | **Are the heliosphere, very local interstellar medium, and local cavity in  pressure balance with Galactic gravity**  |
|| <mark>Jeffrey L. Linsky</mark>, Eberhard Moebius |
|*Appeared on*| *2022-12-02*|
|*Comments*| *15 pages, 1 figure, Astrophysical Journal in press*|
|**Abstract**| The Voyager spacecraft are providing the first in situ measurements of physical properties in the outer heliosphere beyond the heliopause. These data, together with data from the IBEX and HST spacecraft and physical models consistent with these data, now provide critical measurements of pressures in the heliosphere and surrounding interstellar medium. Using these data, we assemble the first comprehensive survey of total pressures inside and outside of the heliopause, in the interstellar gas surrounding the heliosphere, and in the surrounding Local Cavity to determine whether the total pressures in each region are in balance with each other and with the gravitational pressure exerted by the Galaxy. We inter-compare total pressures in each region that include thermal, non-thermal, plasma, ram, and magnetic pressure components. An important result is the role of dynamic (ram) pressure. Total pressure balance at the heliopause can only be maintained with a substantial contribution of dynamic pressure from inside. Also, total pressure balance between the outer heliosphere and pristine very local ISM (VLISM) and between the pristine VLISM and the Local Cavity requires large dynamic pressure contributions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.00130-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.00130) | **PHANGS-JWST First Results: The Influence of Stellar Clusters on PAHs in  Nearby Galaxies**  |
|| Daniel A. Dale, et al. -- incl., <mark>Kiana F. Henny</mark>, <mark>Kathryn Kreckel</mark>, <mark>Janice C. Lee</mark>, <mark>Daizhong Liu</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-12-02*|
|*Comments*| *8 pages, accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ*|
|**Abstract**| We present a comparison of theoretical predictions of dust continuum and polycyclic aromatic hydrocarbon (PAH) emission with new JWST observations in three nearby galaxies: NGC 628, NGC 1365, and NGC 7496. Our analysis focuses on a total of 1063 compact stellar clusters and 2654 stellar associations previously characterized by HST in the three galaxies. We find that the distributions and trends in the observed PAH-focused infrared colors generally agree with theoretical expectations, and that the bulk of the observations is more aligned with models of larger, ionized PAHs. These JWST data usher in a new era of probing interstellar dust and studying how the intense radiation fields near stellar clusters and associations play a role in shaping the physical properties of PAHs. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure figures/seds.pdf</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.00032.md
    + _build/html/tmp_2212.00032/./jwst_muse_alma.png
    + _build/html/tmp_2212.00032/./polar_unwrap.png
    + _build/html/tmp_2212.00032/./timescales.png
exported in  _build/html/2212.00047.md
    + _build/html/tmp_2212.00047/./Scheme.png
    + _build/html/tmp_2212.00047/./proper_motion_dist.png
    + _build/html/tmp_2212.00047/./CMD.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\mpia}{Max-Planck-Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\oxford}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\mcmaster}{Department of Physics and Astronomy, McMaster University, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\cita}{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ghent}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\oan}{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ari}{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\anu}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\ita}{Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r theoretische Astrophysik, Albert-Ueberle-Str. 2, 69120 Heidelberg, Germany}$
$\newcommand{\mpe}{Max-Planck-Institut f\"{u}r extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\cool}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\manch}{Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, University of Manchester, Oxford Road, Manchester M13 9PL, UK}$
$\newcommand{\eso}{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\newcommand{\cral}{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\newcommand{\cfa}{Center for Astrophysics, Harvard \& Smithsonian, 60 Garden St, Cambridge, MA, United States}$
$\newcommand{\CCAPP}{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\OSU}{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\iwr}{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\UBonn}{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121, Bonn, Germany}$
$\newcommand{\stsci}{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD, USA}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\mpia$}{Max-Planck-Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{$\oxford$}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{$\mcmaster$}{Department of Physics and Astronomy, McMaster University, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{$\cita$}{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\newcommand{$\ghent$}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{$\oan$}{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{$\ari$}{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{$\anu$}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{$\ita$}{Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r theoretische Astrophysik, Albert-Ueberle-Str. 2, 69120 Heidelberg, Germany}$
$\newcommand{$\mpe$}{Max-Planck-Institut f\"{u}r extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{$\cool$}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{$\manch$}{Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, University of Manchester, Oxford Road, Manchester M13 9PL, UK}$
$\newcommand{$\eso$}{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\newcommand{$\cral$}{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\newcommand{$\cfa$}{Center for Astrophysics, Harvard \& Smithsonian, 60 Garden St, Cambridge, MA, United States}$
$\newcommand{$\CCAPP$}{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{$\OSU$}{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{$\iwr$}{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{$\UBonn$}{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121, Bonn, Germany}$
$\newcommand{$\stsci$}{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD, USA}$</div>



<div id="title">

#  Reveals Localised Star Formation in a Spiral Arm Spur of NGC 628

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.00032-b31b1b.svg)](https://arxiv.org/abs/2212.00032)<mark>Appeared on: 2022-12-02</mark> - _11 pages, 5 Figures, accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ_

</div>
<div id="authors">

Thomas G. Williams, et al. -- incl., <mark>Eva Schinnerer</mark>, <mark>Jonathan D. Henshaw</mark>, <mark>Frank Bigiel</mark>, <mark>Kathryn Kreckel</mark>, <mark>Daizhong Liu</mark>, <mark>Francesca Pinna</mark>, <mark>Rowan Smith</mark>, <mark>Sophia Stuber</mark>

</div>
<div id="abstract">

**Abstract:** We combine\textit{JWST}observations with ALMA CO and VLT-MUSE H$\alpha$data to examine off-spiral arm star formation in the face-on, grand-design spiral galaxy NGC 628. We focus on the northern spiral arm, around a galactocentric radius of 3--4 kpc, and study two spurs. These form an interesting contrast, as one is CO-rich and one CO-poor, and they have a maximum azimuthal offset in MIRI 21$\mu$m and MUSE H$\alpha$of around 40$^\circ$(CO-rich) and 55$^\circ$(CO-poor) from the spiral arm. The star formation rate is higher in the regions of the spurs near to spiral arms, but the star formation efficiency appears relatively constant. Given the spiral pattern speed and rotation curve of this galaxy and assuming material exiting the arms undergoes purely circular motion, these offsets would be reached in 100 -- 150 Myr, significantly longer than the 21$\mu$m and H$\alpha$star formation timescales (both\textless10 Myr). The invariance of the star formation efficiency in the spurs versus the spiral arms indicates massive star formation is not only triggered in spiral arms, and cannot simply occur in the arms and then drift away from the wave pattern. These early{\it JWST}results show that in-situ star formation likely occurs in the spurs, and that the observed young stars are not simply the `leftovers' of stellar birth in the spiral arms. The excellent physical resolution and sensitivity that{\it JWST}can attain in nearby galaxies will well resolve individual star-forming regions and help us to better understand the earliest phases of star formation.

</div>

<div id="div_fig1">

<img src="tmp_2212.00032/./jwst_muse_alma.png" alt="Fig2" width="100%"/>

**Figure 2. -** Three-colour image of NGC 628, with ALMA CO in blue, MUSE H$\alpha$ in green, and {\it JWST} 21$\mu$m in red. The blue, red, and green boxes show the extent of each corresponding observation. The two spurs we focus on are shown as cyan (CO-rich), and orange (CO-poor) contours (see \S\ref{sec:spur_offset_timescale}). Also shown are the spiral arms from the environmental mask, in gray. The position angle (21$^\circ$; corresponding to $\theta = 0^\circ$ in Figure \ref{fig:polar_unwrap}) is indicated in the lower-right, and a 1 kpc scalebar is shown in the lower-left. (*fig:jwst_muse_alma*)

</div>
<div id="div_fig2">

<img src="tmp_2212.00032/./polar_unwrap.png" alt="Fig4" width="100%"/>

**Figure 4. -** Polar deprojection of NGC 628 in CO (top), 21$\mu$m (middle), and MUSE H$\alpha$(bottom). $0^\circ$ is defined as the position angle (see Fig. \ref{fig:jwst_muse_alma}), and $\theta$ increases from left to right. The nominal co-rotation radius from \cite{2021Williams} is shown as a horizontal dashed white line. We show the approximate ridge of three spiral arms as red lines (determined from the CO). We note there is a clear offset between this CO ridge and the 21$\mu$m/H$\alpha$ ridges \citep[see also, e.g.][]{2018Kreckel} The cyan contour indicates the `CO-rich' spur, and the orange the `CO-poor' spur. (*fig:polar_unwrap*)

</div>
<div id="div_fig3">

<img src="tmp_2212.00032/./timescales.png" alt="Fig1" width="100%"/>

**Figure 1. -** Offset timescale (see Eq. \ref{eq:timescale}) as a function of galactocentric radius for the CO-rich (blue) and CO-poor (red) spurs, calculated between a galactocentric radius of 3 and 4 kpc. (*fig:timescales*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$</div>



<div id="title">

# A first glimpse at the nuclear stellar disc line-of-sight structure

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.00047-b31b1b.svg)](https://arxiv.org/abs/2212.00047)<mark>Appeared on: 2022-12-02</mark> - _Accepted for publication in A&A. 7 pages, 6 figures_

</div>
<div id="authors">

<mark>F. Nogueras-Lara</mark>

</div>
<div id="abstract">

**Abstract:** The nuclear stellar disc (NSD) is a dense stellar structure at the centre of our Galaxy. Given its proximity, it constitutes a unique laboratory to understand other galactic nuclei. Nevertheless, the high crowding and extinction hamper its study, and even its morphology and kinematics are not yet totally clear.In this work we use NSD red clump stars, whose intrinsic properties are well known, to trace the kinematics of the NSD and to compute the distance and extinction towards the edges of the NSD.We used publicly available proper motion and photometric catalogues of the NSD to distinguish red clump stars by using a colour-magnitude diagram. We then applied a Gaussian mixture model to obtain the proper motion distribution, and computed the extinction and distance towards stars with different kinematics.We obtained that the proper motion distributions contain NSD stars rotating eastwards and westwards, plus some contamination from Galactic bulge/bar stars, in agreement with previous work. We computed the distance and extinction towards the eastward- and westward-moving stars and concluded that the latter are$\sim300$pc beyond, indicating a similar structure along and across the line of sight, and  consistent with an axisymmetric structure of the NSD. Moreover, we found that the extinction within the NSD is relatively low and  accounts for less than 10\%of the total extinction of the stars belonging to the farthest edge of the NSD.

</div>

<div id="div_fig1">

<img src="tmp_2212.00047/./Scheme.png" alt="Fig5" width="100%"/>

**Figure 5. -** GALACTICNUCLEUS false colour image of the central region of the NSD. The white rectangles indicate the regions covered by the proper motion catalogue. The white dashed circle shows the effective radius of the nuclear star cluster \citep[$\sim 5$ pc, e.g.][]{gallego-cano2019} with Sagittarius A* at its centre. The white dashed lines indicate the four regions used for the distance and extinction analysis in Sect. \ref{struct}. (*GNS*)

</div>
<div id="div_fig2">

<img src="tmp_2212.00047/./proper_motion_dist.png" alt="Fig2" width="100%"/>

**Figure 2. -** GMM analysis of the proper motion distributions for all the RC stars in the analysed region. The left and right panels show the distribution of the proper motion components parallel ($\mu_l$) and perpendicular ($\mu_b$) to the Galactic plane, respectively. The blue histograms correspond to the data, whereas the orange lines are the total models (solid lines), and each of their Gaussian components (dashed lines). The grey and red shaded areas in the left panel, and the green shaded area in the right panel correspond to the regions analysed in Sect. \ref{struct}. (*proper*)

</div>
<div id="div_fig3">

<img src="tmp_2212.00047/./CMD.png" alt="Fig1" width="100%"/>

**Figure 1. -** CMD $K_s$ vs $H-K_s$ for common stars between the proper motion catalogue and the GALACTINUCLEUS survey. The red parallelogram shows the selection of RC stars. (*CMD*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

116  publications in the last 7 days.
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers